# Indicateurs et ratios

## Calcul des indicateurs et ratios

In [8]:
import pandas as pd
import yaml

In [9]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

DATA_FOLDER = config["DATA_FOLDER"]

In [10]:
revenus_depenses = pd.read_csv(DATA_FOLDER + "revenus-depenses.csv")
revenus_depenses["Montant"] = pd.to_numeric(revenus_depenses["Montant"])
revenus_depenses = revenus_depenses.sort_values(by=["Montant"], ascending=False)

In [11]:
patrimoine = pd.read_csv(DATA_FOLDER + "patrimoine.csv")
patrimoine["Valorisation"] = pd.to_numeric(patrimoine["Valorisation"])

In [12]:
depense = revenus_depenses.loc[revenus_depenses["Type"] == "depense"]
depense_total = depense["Montant"].sum()

In [13]:
revenu_total = revenus_depenses.loc[
    revenus_depenses["Type"] == "revenu", "Montant"
].sum()
epargne = revenu_total - depense_total
taux_depargne = epargne / revenu_total

pourcentage_don = depense["Montant"].values[1] / revenu_total
ratio_liquidite = (
    patrimoine.loc[patrimoine["Type"] == "livret", "Valorisation"].sum() / depense_total
)

taux_depense_variable = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] == "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)
taux_depense_mensuelle = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] != "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)

In [ ]:
print("Taux d'épargne: {:.2%}".format(taux_depargne))
print("Taux de dépense variable: {:.2%}".format(taux_depense_variable))
print("Taux de dépense mensuelle: {:.2%}".format(taux_depense_mensuelle))
print("Pourcentage de don: {:.2%}".format(pourcentage_don))
print("Ratio de liquidité: {:.2}".format(ratio_liquidite))